# Tarea 1
Dado el corpus proporcionado, obtener la curva de Zipf de las palabras en
escala logaritmica.
Pasos:

1. Limpiar el corpus: eliminar signos de puntuacion, de interrogacion, admiracion y elementos no lexicos.

2. Aplicar un algoritmo de Stemming a los tokens limpios.
3. Obtener las frecuencias de los tipos en el corpus.
4. Ordenar por el rango estadistico de mayor a menor.
5. Graficar el diagrama de dispersion rango-frecuencia en escala logarıtmica.

## Importamos el corpus
Lo primero que hacemos es importar el corpus al notebook para que podamos utilizarlos.